In [ ]:
import os
os.chdir("../")

In [ ]:
import pandas as pd

In [ ]:
base_rsna_path = "data/rsna-bone-age/validation"

In [ ]:
test_df = pd.read_csv(os.path.join(base_rsna_path, "Validation Dataset.csv")).rename(columns={"Image ID": "id", "Bone Age (months)": "boneage"})

In [ ]:
test_df['gender'] = test_df['male'].map(lambda x: 0 if x else 1)

In [ ]:
# create path column but map it only if the path exists
test_df["path"] = test_df["id"].map(lambda x: os.path.join(base_rsna_path, "boneage-validation-dataset-1", f"{x}.png") if os.path.exists(os.path.join(base_rsna_path, "boneage-validation-dataset-1", f"{x}.png")) else os.path.join(base_rsna_path, "boneage-validation-dataset-2", f"{x}.png"))


In [ ]:
test_df['exists'] = test_df['path'].map(os.path.exists)

In [ ]:
test_df.drop(columns=['male', 'exists'], inplace=True)

In [ ]:
test_df.columns

In [ ]:
len(test_df)

In [ ]:
test_df["path"] = test_df["id"].map(lambda x: os.path.join(base_rsna_path, "preprocessed", f"{x}.png"))

In [ ]:
test_df['exists'] = test_df['path'].map(os.path.exists)

In [ ]:
test_df = test_df[test_df['exists']]


In [ ]:
len(test_df)

In [ ]:
test_df.to_csv(os.path.join(base_rsna_path, "test_df.csv"))


### Evaluating the results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
test_results = pd.read_csv("output/inception_gender_validation.csv")

In [ ]:
def generate_bins(df):
    num_bins = 10

    # Calculate the bin width
    bin_width = (df['actual_ages'].max() - df['actual_ages'].min()) / num_bins

    # Create the bin boundaries
    bin_boundaries = [df['actual_ages'].min() + i * bin_width for i in range(num_bins + 1)]

    # Create the labels for the bins
    bin_labels = [f'Bin {i+1}' for i in range(num_bins)]

    # Add a new column to the dataframe representing the bin labels
    df['bin'] = pd.cut(df['actual_ages'], bins=bin_boundaries, labels=bin_labels, include_lowest=True)

    # Create a dictionary to store the separate dataframes
    dfs = {}

    # Split the dataframe into separate dataframes based on the bin labels
    for label in bin_labels:
        dfs[label] = df[df['bin'] == label].copy()

    for label, df_bin in dfs.items():
        df_bin.drop('bin', axis=1, inplace=True)

    return dfs

In [ ]:
def plot_mae(dfs):
    mae_values = {}

    # Create a dictionary to store the bin ranges and counts
    bin_ranges = {}
    bin_counts = {}

    # Calculate the MAE for each bin and store the bin ranges and counts
    for label, df_bin in dfs.items():
        actual_ages = df_bin['actual_ages']
        predicted_ages = df_bin['predicted_ages']
        mae = (actual_ages - predicted_ages).abs().mean()
        mae_values[label] = mae
        bin_ranges[label] = f'{df_bin["actual_ages"].min()}-{df_bin["actual_ages"].max()}'
        bin_counts[label] = len(df_bin)

    # Convert the MAE values, bin ranges, and counts to pandas Series
    mae_series = pd.Series(mae_values)
    bin_ranges_series = pd.Series(bin_ranges)
    bin_counts_series = pd.Series(bin_counts)

    # Plot the MAE values
    plt.figure(figsize=(10, 6))
    ax = mae_series.plot(kind='bar')
    plt.xlabel('Bins')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.title('MAE for Each Bin')
    plt.xticks(range(len(bin_ranges)), [f'{bin_ranges_series[i]}\nCount: {bin_counts_series[i]}' for i in range(len(bin_ranges))], rotation=45)
    plt.show()

In [ ]:
dfs = generate_bins(test_results[test_results['genders']==0])
plot_mae(dfs)

In [ ]:
dfs = generate_bins(test_results[test_results['genders']==1])
plot_mae(dfs)